In [2]:
import pandas as pd
import numpy as np


In [3]:
data_path = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00583/Chemical%20Composion%20of%20Ceramic.csv'
df = pd.read_csv(data_path)


In [5]:
df


,Ceramic Name,Part,Na2O,MgO,Al2O3,SiO2,K2O,CaO,TiO2,Fe2O3,MnO,CuO,ZnO,PbO2,Rb2O,SrO,Y2O3,ZrO2,P2O5
0,FLQ-1-b,Body,0.62,0.38,19.61,71.99,4.84,0.31,0.07,1.18,630,10,70,10,430,0,40,80,90
1,FLQ-2-b,Body,0.57,0.47,21.19,70.09,4.98,0.49,0.09,1.12,380,20,80,40,430,-10,40,100,110
2,FLQ-3-b,Body,0.49,0.19,18.60,74.70,3.47,0.43,0.06,1.07,420,20,50,50,380,40,40,80,200
3,FLQ-4-b,Body,0.89,0.30,18.01,74.19,4.01,0.27,0.09,1.23,460,20,70,60,380,10,40,70,210
4,FLQ-5-b,Body,0.03,0.36,18.41,73.99,4.33,0.65,0.05,1.19,380,40,90,40,360,10,30,80,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,DY-M-3-g,Glaze,0.34,0.55,12.37,70.70,5.33,8.06,0.06,1.61,1250,10,90,30,250,520,30,140,690
84,DY-QC-1-g,Glaze,0.72,0.34,12.20,72.19,6.19,6.06,0.04,1.27,1700,60,110,10,270,540,40,120,630
85,DY-QC-2-g,Glaze,0.23,0.24,12.99,71.81,5.25,7.15,0.05,1.29,750,40,100,0,240,470,40,120,480
86,DY-QC-3-g,Glaze,0.14,0.46,12.62,69.16,4.34,11.03,0.05,1.20,920,40,90,20,230,470,40,130,1100


In [6]:
X = df.drop(columns=['Ceramic Name', 'Part'])
X = X.values
print(X)


[[6.200e-01 3.800e-01 1.961e+01 ... 4.000e+01 8.000e+01 9.000e+01]
 [5.700e-01 4.700e-01 2.119e+01 ... 4.000e+01 1.000e+02 1.100e+02]
 [4.900e-01 1.900e-01 1.860e+01 ... 4.000e+01 8.000e+01 2.000e+02]
 ...
 [2.300e-01 2.400e-01 1.299e+01 ... 4.000e+01 1.200e+02 4.800e+02]
 [1.400e-01 4.600e-01 1.262e+01 ... 4.000e+01 1.300e+02 1.100e+03]
 [1.400e-01 6.300e-01 1.425e+01 ... 4.000e+01 1.100e+02 6.900e+02]]


In [7]:
def kmeanspp(X, ncls):
    ndata = X.shape[0]
    nfeature = X.shape[1]
    centroid = np.zeros((ncls, nfeature))
    centroid[0, :] = X[np.random.randint(ndata), :]

    for icls in range(1, ncls):
        sum = 0
        distance = np.zeros((ncls, ndata))
        for cls in range(icls):
            for idata in range(ndata):
                for ifeature in range(nfeature):
                    distance[cls][idata] += (X[idata][ifeature] -
                                             centroid[cls][ifeature]) ** 2

        res_distance = np.zeros(ndata)

        for idata in range(ndata):
            min_val = 0
            for cls in range(icls):
                if (cls == 0):
                    min_val = distance[cls][idata]
                    res_distance[idata] = min_val
                elif (distance[cls][idata] < min_val):
                    min_val = distance[cls][idata]
                    res_distance[idata] = min_val

        for idata in range(ndata):
            sum += res_distance[idata]

        rnd = np.random.rand() * sum
        s = 0
        cdata = 0
        while (s < rnd):
            s += res_distance[cdata]
            cdata += 1

        centroid[icls] = X[cdata - 1]

    for idata in range(ndata):
        for ifeature in range(nfeature):
            distance[icls][idata] += (X[idata][ifeature] -
                                      centroid[icls][ifeature]) ** 2

    dist_cls = np.zeros(ndata)

    for idata in range(ndata):
        min_val = 0
        for cls in range(ncls):
            if cls == 0:
                min_val = distance[cls][idata]
                dist_cls[idata] = cls
                res_distance[idata] = min_val
            elif distance[cls][idata] < min_val:
                min_val = distance[cls][idata]
                dist_cls[idata] = cls
                res_distance[idata] = min_val

    return centroid, dist_cls


def center(X, dist_cls, ncls):
    nfeature = X.shape[1]
    centroid = np.zeros((ncls, nfeature))
    for icls in range(ncls):
        indices = np.where(dist_cls == icls)
        n = indices[0].size
        for id in indices[0]:
            for ifeature in range(nfeature):
                centroid[icls][ifeature] += X[id][ifeature] / n
    return centroid


def kmeans(X, ncls, iter_max):
    ndata = X.shape[0]
    nfeature = X.shape[1]
    distance = np.zeros((ncls, ndata))
    centroid, dist_cls = kmeanspp(X, ncls)
    stabilization = False
    iter = 0
    while not stabilization and iter < iter_max:
        new_centroid = center(X, dist_cls, ncls)
        for cls in range(ncls):
            for idata in range(ndata):
                for ifeature in range(nfeature):
                    distance[cls][idata] += (X[idata][ifeature] -
                                             new_centroid[cls][ifeature]) ** 2
        for idata in range(ndata):
            min_val = 0
            for cls in range(ncls):
                if (cls == 0):
                    min_val = distance[cls][idata]
                    dist_cls[idata] = cls
                elif (distance[cls][idata] < min_val):
                    min_val = distance[cls][idata]
                    dist_cls[idata] = cls

        if (np.array_equal(new_centroid, centroid)):
            stabilization = True
        else:
            centroid = new_centroid
        iter += 1
    return centroid, dist_cls


In [11]:
centroid, dist_cls = kmeans(X, 2, 1000)
dist_cls

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0.])

In [9]:
def shadow(X, dist_cls):
    ndata = X.shape[0]
    nfeature = X.shape[1]
    s_data = np.zeros(ndata)
    a_data = np.zeros(ndata)
    b_data = np.zeros(ndata)
    for idata in range(ndata):
        cls = dist_cls[idata]
        indices = np.where(dist_cls == cls)
        min_distance = float('inf')
        min_cls = 0
        for id in indices[0]:
            for ifeature in range(nfeature):
                a_data += (X[idata][ifeature] - X[id][ifeature]) ** 2

        for id in range(ndata):
            if id not in indices[0]:
                distance = 0
                for ifeature in range(nfeature):
                    distance += (X[idata][ifeature] - X[id][ifeature]) ** 2
                distance = np.sqrt(distance)
                if distance < min_distance:
                    min_distance = distance
                    min_cls = dist_cls[id]

        indices_min = np.where(dist_cls == min_cls)
        for id in indices_min[0]:
            for ifeature in range(nfeature):
                b_data += (X[idata][ifeature] - X[id][ifeature]) ** 2

        a_data[idata] = np.sqrt(a_data[idata]) / len(indices[0])
        b_data[idata] = np.sqrt(b_data[idata]) / len(indices_min[0])
        s_data[idata] = (b_data[idata] - a_data[idata]) / \
            max(a_data[idata], b_data[idata])

    s_avg = sum(s_data) / ndata

    return s_avg


In [12]:
shadow(X, dist_cls)

0.6499302438373825

In [13]:
def dbscan(X, epsilon, min_pts):
    ndata = X.shape[0]
    nfeature = X.shape[1]
    distance = np.zeros((ndata, ndata))
    dist_cls = np.zeros(ndata)
    for idata in range(ndata-1):
        for jdata in range(idata+1, ndata):
            for ifeature in range(nfeature):
                distance[idata][jdata] += (X[idata]
                                           [ifeature] - X[jdata][ifeature]) ** 2
            distance[idata][jdata] = np.sqrt(distance[idata][jdata])
            distance[jdata][idata] = distance[idata][jdata]
    pts = np.zeros(ndata)
    k = 0
    while 0 in pts:
        unverified_points = np.where(pts == 0)[0]
        pt = np.random.choice(unverified_points)
        pts[pt] = 1
        cl = [pt]
        for idata in range(ndata):
            if distance[pt][idata] < epsilon and not idata in cl and pts[idata] == 0:
                cl.append(idata)
        for i in range(len(cl)):
            for idata in range(ndata):
                if distance[cl[i]][idata] < epsilon and not idata in cl and pts[idata] == 0:
                    cl.append(idata)
            pts[cl[i]] = 1
        if len(cl) >= min_pts: 
            k += 1
            for dist in cl:
                dist_cls[dist] = k

    return dist_cls


In [36]:
cls = dbscan(X, 800, 3)
cls

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 3., 3., 3., 2.,
       3., 1., 3., 1., 1., 1., 2., 2., 2., 3., 2., 2., 2., 3., 3., 1., 2.,
       2., 2., 2., 2., 2., 2., 1., 1., 2., 2., 3., 2., 3., 0., 2., 2., 2.,
       1., 3., 3.])

In [37]:
shadow(X, cls)

0.631803742813338